In [210]:
import pandas as pd
from bs4 import BeautifulSoup 
import requests
import re
import sys

In [211]:
oslo = "0.20061"
viken = "0.22030"
vestfold_telemark = "0.22038"
asker = "1.22030.20046"
lier = "1.22030.20125"
drammen ="1.22030.20110"


In [212]:
def get_data(by, pages): 
    columns = ["pris", "tittel", "kvadratmeter", "adresse", "sted", "totalpris","fellesutgifter","eierform","soverom",  "type", "link" ]
    df = pd.DataFrame(columns=columns)
    pages = int(pages)
    for i in range(1,pages):
        url = r"https://www.finn.no/realestate/homes/search.html?location={}&page={}&sort=PUBLISHED_DESC"
        response = requests.get(url.format(by, i))
        #print(response)
        #print(url.format(oslo, i))
        soup = BeautifulSoup(response.content,"html.parser")
        tittel = soup.find_all(class_="ads__unit__link")


        for p in range(1,len(tittel)):
            tittel = soup.find_all(class_="ads__unit__link")[p].text
            adresse = soup.find_all(class_="ads__unit__content__details")[p].text
            kvmpris=  soup.find_all(class_="ads__unit__content__keys")[p].text
            kvm = kvmpris[:kvmpris.find("m")]
            link = soup.find_all(class_="ads__unit__link", href=True)[p]["href"]
            pris_skitten = kvmpris[5:]
            pris = re.sub("[^0-9]", "", pris_skitten)
            sted = adresse[adresse.find(","):]
            #nye variabler
            info = soup.find_all(class_="u-float-left")[p].text
            variablecount = info.count("•")  
            if variablecount == 3: 
              totalpris_skitten, fellesutg, tyype, antsoverom =  info.split("•")
            totalpris = re.sub("[^0-9]", "", totalpris_skitten)
            soverom = re.sub("[^0-9]", "", antsoverom)
            disallowed_characters = "xa"
            for character in disallowed_characters: 
                fellesutg_ren = fellesutg.replace(character, "")

            fellesutg = re.sub("[^0-9]", "", fellesutg_ren)
            eierform = fellesutg_ren[fellesutg_ren.find("kr"):].replace("kr","")
            i = i+1
            df =df.append( {
                    "pris":pris, 
                    "tittel":tittel,
                    "kvadratmeter":kvm,
                    "pris":pris,
                    "adresse":adresse, 
                    "fylke":by,
                    "totalpris":totalpris,
                    "fellesutgifter":fellesutg,
                    "soverom":soverom,
                    "eierform":eierform, 
                    "type":tyype,
                    "link":link
                                    
                    },ignore_index=True)
    return df

In [213]:
df = get_data(oslo,2) #tester
df

,pris,tittel,kvadratmeter,adresse,sted,totalpris,fellesutgifter,eierform,soverom,type,link,fylke
0,9300000,Høyt og luftig på Ensjø Torg - Nyere 4-roms me...,100,"Rolf Hofmos gate 32, Oslo",NaN,9535592,4613,Eier (Selveier),3,Leilighet,https://www.finn.no/realestate/homes/ad.html?f...,0.20061
1,2990000,Grorud - Innbydende 2-roms selveier med trappe...,48,"Sagstuveien 5 C, Oslo",NaN,3066092,2395,Eier (Selveier),1,Leilighet,https://www.finn.no/realestate/homes/ad.html?f...,0.20061
2,10900000,Besserud - Lun og lekker leilighet med trafikk...,103,"Kragstubben 32, Oslo",NaN,11190633,3801,Eier (Selveier),2,Leilighet,https://www.finn.no/realestate/homes/ad.html?f...,0.20061
3,5500000,Frysja - Lys og attraktiv 3-roms med fantastis...,73,"Frysjaveien 3, Oslo",NaN,5952516,5255,Eier (Selveier),2,Leilighet,https://www.finn.no/realestate/homes/ad.html?f...,0.20061
4,7950000,NYDALEN - Delikat og gjennomgående 4-roms leil...,80,"Nydalsveien 22B, Oslo",NaN,8150092,3697,Eier (Selveier),3,Leilighet,https://www.finn.no/realestate/homes/ad.html?f...,0.20061
5,4690000,"HØYBRÅTEN - Delikat, gjennomgående 3-roms med ...",93,"Starveien 18A, Oslo",NaN,5089526,5060,Andel,2,Leilighet,https://www.finn.no/realestate/homes/ad.html?f...,0.20061
6,6100000,Hellerudtoppen - Fin og gjennomgående 4-R ende...,82,"Landerokollen 46, Oslo",NaN,6316702,3678,Eier (Selveier),3,Leilighet,https://www.finn.no/realestate/homes/ad.html?f...,0.20061
7,7800000,"Lysejordet / Ullern - Ullern/Lysejordet, Lekke...",104,"Møllefaret 62A, Oslo",NaN,7830554,4291,Andel,3,Leilighet,https://www.finn.no/realestate/homes/ad.html?f...,0.20061
8,10490000,Majorstuen / Homansbyen - Rålekker 4(5)-roms l...,92,"Oscars gate 8C, Oslo",NaN,11004547,5539,Aksje,3,Leilighet,https://www.finn.no/realestate/homes/ad.html?f...,0.20061
9,6300000,"Bjørndal - Lekker, kjedet enebolig over 3 plan...",141,"Langteigåsen 27, Oslo",NaN,6473170,1022,Eier (Selveier),3,Enebolig,https://www.finn.no/realestate/homes/ad.html?f...,0.20061


In [214]:
len(df.tittel.unique()) #no duplicates

50

In [215]:
df.shape

(50, 12)

In [216]:
#making the bigger dataset
df1 = get_data(oslo,50)
df2 = get_data(viken,50)
df3 = get_data(vestfold_telemark,50)

In [217]:
df1["fylke"] = "oslo"
df2["fylke"] = "viken"
df3["fylke"] = "vestfold og telemark"
frames = [df1,df2,df3]
result = pd.concat(frames)

In [218]:
result.to_excel("data.xlsx")

In [219]:
len(result.tittel.unique())

3509

In [220]:
result.shape #might be because there are some duplicates, but I know for a fact that some companies etc make several cards with the exact same name when they 
#are promoting new buildings etc

(3543, 12)

In [221]:
result

,pris,tittel,kvadratmeter,adresse,sted,totalpris,fellesutgifter,eierform,soverom,type,link,fylke
0,9300000,Høyt og luftig på Ensjø Torg - Nyere 4-roms me...,100,"Rolf Hofmos gate 32, Oslo",NaN,9535592,4613,Eier (Selveier),3,Leilighet,https://www.finn.no/realestate/homes/ad.html?f...,oslo
1,2990000,Grorud - Innbydende 2-roms selveier med trappe...,48,"Sagstuveien 5 C, Oslo",NaN,3066092,2395,Eier (Selveier),1,Leilighet,https://www.finn.no/realestate/homes/ad.html?f...,oslo
2,10900000,Besserud - Lun og lekker leilighet med trafikk...,103,"Kragstubben 32, Oslo",NaN,11190633,3801,Eier (Selveier),2,Leilighet,https://www.finn.no/realestate/homes/ad.html?f...,oslo
3,5500000,Frysja - Lys og attraktiv 3-roms med fantastis...,73,"Frysjaveien 3, Oslo",NaN,5952516,5255,Eier (Selveier),2,Leilighet,https://www.finn.no/realestate/homes/ad.html?f...,oslo
4,7950000,NYDALEN - Delikat og gjennomgående 4-roms leil...,80,"Nydalsveien 22B, Oslo",NaN,8150092,3697,Eier (Selveier),3,Leilighet,https://www.finn.no/realestate/homes/ad.html?f...,oslo
...,...,...,...,...,...,...,...,...,...,...,...,...
631,1900000,Koslig leilighet med høy standard - alt på en ...,72,"Tøddelia 1, Fyresdal",NaN,1949014,850,Eier (Selveier),2,Leilighet,https://www.finn.no/realestate/homes/ad.html?f...,vestfold og telemark
632,1790000,HERRE - BAMBLE - Utsiktseiendom ved fjorden. ...,120,"Hellestvedtveien 67, Herre",NaN,1949014,850,0,2,Leilighet,https://www.finn.no/realestate/homes/ad.html?f...,vestfold og telemark
633,3654300,Holmestrand/Skogstjerna - Ny 3-roms i 2. etg. ...,88,"Skogstjernen 14, Holmestrand",NaN,3670300,2000,Eier (Selveier),2,Leilighet,https://www.finn.no/realestate/homes/ad.html?f...,vestfold og telemark
634,900000,Utsiktsleilighet i Sandefjord sentrum,92,"Jernbaneplassen 1, Sandefjord",NaN,1112000,5342,Obligsjon,2,Leilighet,https://www.finn.no/realestate/homes/ad.html?f...,vestfold og telemark
